<a href="https://colab.research.google.com/github/AlexArq/AlexArq/blob/TFM/RANDOM_FOREST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LIBRERÍAS

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pandas import read_csv
from timeit import default_timer as timer
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### DATASET

In [ ]:
data_dir = os.path.dirname('/content/drive/My Drive/TFM/')
lista = os.listdir(data_dir)
lista

['VENTANA_G3 (1).csv',
 'VENTANA_G3.csv',
 'VENTANA_G2.csv',
 'VENTANA_G2 (1).csv',
 'VENTANA_G3_wauh.csv']

### ENTRENAMIENTO

In [ ]:
for in range (0, len(lista)):
  start = timer()
  dataset = pd.read_csv(data_dir+'/'+str(lista[i]),sep=',', decimal = '.',na_values= '')
  dataset = dataset.fillna(dataset.mean())
  atributos = []
  for var in list(dataset.columns):
    if var[0].isdigit():
      atributos.append(var)
      atributos.append('CLASE')
  variables = dataset.loc[:,atributos]
  X = np.array(variables.loc[:,'1_mean':'11_majority'])
  Y = np.array(variables.loc[:,'CLASE'])
  scaler = StandardScaler()
  np.random.seed(12)
  X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, train_size=0.7)
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  np.random.seed(12)
  scoring = {'AUC': 'roc_auc', 'F1 score': 'f1', 'Precision': 'precision', 'Recall': 'recall', 'B_Accuracy': 'balanced_accuracy'}
  model = RandomForestClassifier(random_state=98,
                               min_samples_leaf = 1,
                               min_samples_split = 2,                                       
                               n_jobs = -1
                               ) 
  param_grid = {'criterion': ['entropy', 'gini'],
              'n_estimators':[10, 50,100,150,200], 
              'max_features':['sqrt','log2',None]}

  RF = model_selection.GridSearchCV(model, param_grid, scoring= scoring, refit = scoring.values[i], cv=10, return_train_score=True)
  RF = RF.fit(X_train, y_train)
  
  RF_Train =RF.predict(X_train)
  gmTrain = geometric_mean_score(y_train,RF_Train)

  RF_Test = RF.predict(X_test)
  gmTest = geometric_mean_score(y_test, RF_Test)

  cl_probs = RF.predict_proba(X_test)
  cl_probs = cl_probs[:, 1]
  cl_auc = metrics.roc_auc_score(y_test, cl_probs)
  lr_f1 = metrics.f1_score(y_test, RF_Test)

  cm = confusion_matrix(y_test, RF_Test, labels=RF.classes_)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,
  display_labels=RF.classes_)

  kappa = cohen_kappa_score(y_test, RF_Test)

  indicekappa = ['Pobre','Débil', 'Moderado', 'Bueno', 'Muy Bueno']
  verindicekappa = []
  if kappa < 0.2:
      verindicekappa = indicekappa[0]
  elif kappa > 0.2 and kappa < 0.4:
      verindicekappa = indicekappa[1]
  elif kappa > 0.4 and kappa < 0.6:
      verindicekappa = indicekappa[2]
  elif kappa > 0.6 and kappa < 0.8:
      verindicekappa = indicekappa[3]
  else:
      verindicekappa = indicekappa[4]

  model_probs = RF.predict_proba(X_test)
  model_probs = model_probs[:, 1]

  ns_probs = [0 for _ in range (len (y_test))]
  ns_auc = metrics.roc_auc_score(y_test, ns_probs)*100.0

  model_auc = metrics.roc_auc_score(y_test, model_probs)*100.0
  ns_fpr, ns_tpr, _ = metrics.roc_curve(y_test, ns_probs)
  model_fpr, model_tpr, _ = metrics.roc_curve(y_test, model_probs)

  print('Archivo: {}\nMetodo: {}\nMejores Hiperparámetros: {}\nMejor puntuación: {}\nRendimiento: (GM) {}\nROC AUC: {}\nF1 score: {}\nÍndice de Kappa: {}\nClasificador: {}'.format(lista[i],model,
                                                                                       RF.best_params_, 
                                                                                       round(RF.best_score_,2),
                                                                                       round(gmTest*100.0, 2),
                                                                                       round(cl_auc*100.0, 2), 
                                                                                       round(lr_f1*100.0, 2),
                                                                                       round(kappa,2),
                                                                                       verindicekappa))
  
  
  tiempo_RF_aprender = timer() - start

  print ("Calculados los mejores hiperparámetros en %f segundos" % tiempo_RF_aprender )





